In [ ]:
import os
import anndata as ad
import numpy as np
import scanpy as sc
from PIL import Image

Image.MAX_IMAGE_PIXELS = 1000000000

In [ ]:
def rotate_image(image_path: str, degrees: int, outdir: str):
    im = Image.open(image_path)
    rot_im = im.rotate(-degrees, expand=1)

    suffix = f"-rotated_{degrees}"
    fname, ext = os.path.splitext(os.path.basename(image_path))
    rot_im.save(os.path.join(outdir, fname + suffix + ext))


def rotate_anndata(anndata_path: str, degrees: int, outdir: str):
    adata = ad.read(anndata_path)
    for sample in adata.uns["spatial"].keys():
        hiresfactor = adata.uns["spatial"][sample]["scalefactors"][
            "tissue_hires_scalef"
        ]
        n, m, _ = adata.uns["spatial"][sample]["images"]["hires"].shape
        m = int(m / hiresfactor)
        n = int(n / hiresfactor)

        rot_spatial = []
        for [x, y] in adata.obsm["spatial"]:
            if degrees == 90:
                rot_spatial.append([n - y, x])
            elif degrees == 180:
                rot_spatial.append([m - x, n - y])
            elif degrees == 270:
                rot_spatial.append([y, m - x])

        adata.obsm["spatial"] = np.array(rot_spatial)

        hires = adata.uns["spatial"][sample]["images"]["hires"]
        rot_hires = np.rot90(hires, k=degrees / 90, axes=(1, 0))
        adata.uns["spatial"][sample]["images"]["hires"] = rot_hires

        lowres = adata.uns["spatial"][sample]["images"]["lowres"]
        rot_lowres = np.rot90(lowres, k=degrees / 90, axes=(1, 0))
        adata.uns["spatial"][sample]["images"]["lowres"] = rot_lowres

    suffix = f"-rotated_{degrees}"
    fname, ext = os.path.splitext(os.path.basename(anndata_path))
    adata.write_h5ad(os.path.join(outdir, fname + suffix + ext))


def rotate(
    anndata_path: str = None,
    image_path: str = None,
    degrees: int = 90,
    outdir: str = "./",
):
    if anndata_path:
        rotate_anndata(anndata_path, degrees, outdir)
    if image_path:
        rotate_image(image_path, degrees, outdir)
    return

In [ ]:
outdir = "./"
data = [
    {
        "anndata_path": "/path/to/anndata.h5ad",
        "image_path": "/path/to/image.tiff",
        "degrees": 90, # 90, 180 or 270
    }
]

In [ ]:
for item in data:
    rotate(**item, outdir=outdir)